In [1]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from scipy import stats

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

#from keras_sequential_ascii import keras2ascii
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
ST_code = time.time()
# fix random seed for reproducibility
np.random.seed(51)

In [3]:
#read the data from CSV
df = pd.read_csv("WBG_ADAK.csv")
df

,DO_org,DO_w,DO_b,DO_g,Unnamed: 4,Unnamed: 5,Unnamed: 6,DOE_w,DOE_b,DOE_g
0,7.578252,7.569668,7.611028,7.616017,NaN,NaN,NaN,0.008584,-0.032776,-0.037765
1,7.576823,7.567345,7.608721,7.614087,NaN,NaN,NaN,0.009478,-0.031898,-0.037264
2,7.613978,7.604609,7.645992,7.649406,NaN,NaN,NaN,0.009370,-0.032014,-0.035428
3,7.589685,7.579819,7.621189,7.625750,NaN,NaN,NaN,0.009865,-0.031505,-0.036066
4,7.596830,7.588037,7.629403,7.633458,NaN,NaN,NaN,0.008793,-0.032573,-0.036628
...,...,...,...,...,...,...,...,...,...,...
1091,7.588255,7.578748,7.620060,7.623873,NaN,NaN,NaN,0.009508,-0.031804,-0.035618
1092,7.592543,7.582989,7.624265,7.627402,NaN,NaN,NaN,0.009554,-0.031723,-0.034859
1093,7.596830,7.587358,7.628642,7.631650,NaN,NaN,NaN,0.009472,-0.031812,-0.034821
1094,7.629698,7.620188,7.661476,7.662973,NaN,NaN,NaN,0.009510,-0.031778,-0.033275


In [ ]:

#Hyperparameters verbose,epochs,windowsize,batchsize, shuffle_buffer sieze, learning rate, optimizer
split= 0.8
s = 'S8020'
v= 0
epno = 10
e = '_e100_'
wz = 30
w = 'wz30'
bz = 32
b ='b32_ADAK_ADAM'
shuffle_buffer_size = 1000
lrno = 0.0008
l = 'l0008'
window_size = wz
look_back = wz
batch_size = bz

LOOK_BACK = wz
#FORECAST_RANGE = FR
n_features = 1
epochs = epno
batch_size = bz
validation = 0.2

In [ ]:
FR = 1
FORECAST_RANGE = FR
FRn = '_FR2_'
opt = tf.keras.optimizers.Adam(learning_rate=lrno) #, momentum=0.9)
#opt = tf.keras.optimizers.Adagrad(learning_rate=0.0008,initial_accumulator_value=0.1,epsilon=1e-07,name="Adagrad")
#%run 2.io_Loop_4WQP.ipynb

In [ ]:
WQP = 'SAL'
WQPn = '1.SAL'
WQPu = 'Salinity (ppt)'

# %run 3.io_Models_Order2.ipynb
# df_results_SAL = pd.DataFrame()
# df_results_SAL = df_results

# df_results.to_csv('n/'+ WQPn +'_Results_' + l + s + e + w + b + FRn +'.csv')

# Step = FR
# rmseLSTM, rmseGRU, rmseCNN, rmseCNN_LSTM, rmseCNN_GRU = df_results_SAL.rmse_test
# res = pd.DataFrame([Step, rmseLSTM, rmseGRU, rmseCNN, rmseCNN_LSTM, rmseCNN_GRU], index= ["Step", "LSTM", "GRU", "CNN", "CNN-LSTM", "CNN-GRU"]).T
# df_rmse1_SAL = df_rmse1_SAL.append(res)

# # df_t_SAL = df_results_SAL.time_s
# # res = df_t_SAL.T
# # df_t1_SAL = df_t1_SAL.append(res)

# mae_test, mse_test, rmse_test, mape_test = evaluate_forecast(y_test_inverse, yhat_inverse)

# res = pd.DataFrame([name, mae_test, mse_test, rmse_test, mape_test], index= ["Name", "mae_test", "mse_test", "rmse_test", "mape_test"]).T
# df_results = df_results.append(res)

In [ ]:
#x_train = df[WQP].dropna()
print(WQP)

In [ ]:
df_results = pd.DataFrame()

In [ ]:

values = df[WQP].values
dataset = values

train_size = int(len(dataset) * split)
test_size = len(dataset) - train_size
df_train = dataset[:train_size]
df_test = dataset[train_size:]

In [ ]:
scaler = MinMaxScaler()
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(values[~np.isnan(values)].reshape(-1, 1))
scaled_train = scaler.fit_transform(df_train[~np.isnan(df_train)].reshape(-1, 1))
scaled_test = scaler.transform(df_test[~np.isnan(df_test)].reshape(-1, 1))

In [ ]:
def split_sequence(sequence, look_back, forecast_horizon):
 X, y = list(), list()
 for i in range(len(sequence)): 
   lag_end = i + look_back
   forecast_end = lag_end + forecast_horizon
   if forecast_end > len(sequence):
     break
   seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
   X.append(seq_x)
   y.append(seq_y)
 return np.array(X), np.array(y)

In [ ]:
X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)

In [ ]:
def inverse_transform(y_test, yhat):
 y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
 yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])
 yhat_inverse = scaler.inverse_transform(yhat_reshaped)
 y_test_inverse = scaler.inverse_transform(y_test_reshaped)
 return yhat_inverse, y_test_inverse

In [ ]:
def evaluate_forecast(y_test_inverse, yhat_inverse):
 mse_ = tf.keras.losses.MeanSquaredError()
 mae_ = tf.keras.losses.MeanAbsoluteError()
 mape_ = tf.keras.losses.MeanAbsolutePercentageError()
 mae = mae_(y_test_inverse,yhat_inverse).numpy()
 mse = mse_(y_test_inverse,yhat_inverse).numpy()
 mape = mape_(y_test_inverse,yhat_inverse).numpy()
 rmse = np.sqrt(mse)
 return mae, mse, rmse, mape

In [ ]:
name = "LSTM"
model = tf.keras.models.Sequential([
tf.keras.layers.LSTM(32, input_shape=(LOOK_BACK, n_features)),
tf.keras.layers.RepeatVector(FORECAST_RANGE),
tf.keras.layers.Dense(30, activation="relu"),
tf.keras.layers.Dense(10, activation="relu"),
tf.keras.layers.Dense(FORECAST_RANGE),
tf.keras.layers.Lambda(lambda x: x * 400)  
      ])
model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
#plot_model(model=model, show_shapes=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,verbose=0)#,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
yhat = model.predict(X_test)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
mae_test, mse_test, rmse_test, mape_test = evaluate_forecast(y_test_inverse, yhat_inverse)

res = pd.DataFrame([name, mae_test, mse_test, rmse_test, mape_test], index= ["Name", "mae_test", "mse_test", "rmse_test", "mape_test"]).T
df_results = df_results.append(res)

In [ ]:
name = "GRU"
model = tf.keras.models.Sequential([
tf.keras.layers.GRU(32, input_shape=(LOOK_BACK, n_features)),
tf.keras.layers.RepeatVector(FORECAST_RANGE),
tf.keras.layers.Dense(30, activation="relu"),
tf.keras.layers.Dense(10, activation="relu"),
tf.keras.layers.Dense(FORECAST_RANGE),
tf.keras.layers.Lambda(lambda x: x * 400)  
      ])
model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
#plot_model(model=model, show_shapes=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,verbose=0)#,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
yhat = model.predict(X_test)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
mae_test, mse_test, rmse_test, mape_test = evaluate_forecast(y_test_inverse, yhat_inverse)

res = pd.DataFrame([name, mae_test, mse_test, rmse_test, mape_test], index= ["Name", "mae_test", "mse_test", "rmse_test", "mape_test"]).T
df_results = df_results.append(res)

In [ ]:
name = "CNN"
model = tf.keras.models.Sequential([
tf.keras.layers.Conv1D(32,2, input_shape=(LOOK_BACK, n_features)),
tf.keras.layers.MaxPooling1D(pool_size=1),
#tf.keras.layers.Dropout(0.25),
tf.keras.layers.Flatten(),
tf.keras.layers.RepeatVector(FORECAST_RANGE),
tf.keras.layers.Dense(30, activation="relu"),
tf.keras.layers.Dense(10, activation="relu"),
tf.keras.layers.Dense(FORECAST_RANGE),
tf.keras.layers.Lambda(lambda x: x * 400)  
      ])
model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
#plot_model(model=model, show_shapes=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,verbose=0)#,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
yhat = model.predict(X_test)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
mae_test, mse_test, rmse_test, mape_test = evaluate_forecast(y_test_inverse, yhat_inverse)

res = pd.DataFrame([name, mae_test, mse_test, rmse_test, mape_test], index= ["Name", "mae_test", "mse_test", "rmse_test", "mape_test"]).T
df_results = df_results.append(res)

In [ ]:
name = "CNN-LSTM"
model = tf.keras.models.Sequential([
tf.keras.layers.Conv1D(32,2, activation="relu", input_shape=(LOOK_BACK, n_features)),
tf.keras.layers.MaxPooling1D(pool_size=1),
#tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.RepeatVector(FORECAST_RANGE),
tf.keras.layers.LSTM(32, activation = 'relu', return_sequences=True),
tf.keras.layers.Dense(FORECAST_RANGE)
#tf.keras.layers.Lambda(lambda x: x * 400)  
      ])

model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
#plot_model(model=model, show_shapes=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,verbose=0)#,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
yhat = model.predict(X_test)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
mae_test, mse_test, rmse_test, mape_test = evaluate_forecast(y_test_inverse, yhat_inverse)

res = pd.DataFrame([name, mae_test, mse_test, rmse_test, mape_test], index= ["Name", "mae_test", "mse_test", "rmse_test", "mape_test"]).T
df_results = df_results.append(res)

In [ ]:
name = "CNN-GRU"
model = tf.keras.models.Sequential([
tf.keras.layers.Conv1D(32,2, activation="relu", input_shape=(LOOK_BACK, n_features)),
tf.keras.layers.MaxPooling1D(pool_size=1),
#tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.RepeatVector(FORECAST_RANGE),
tf.keras.layers.GRU(32, activation = 'relu', return_sequences=True),
tf.keras.layers.Dense(FORECAST_RANGE)
#tf.keras.layers.Lambda(lambda x: x * 400)  
      ])

model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
#plot_model(model=model, show_shapes=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,verbose=0)#,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
yhat = model.predict(X_test)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
mae_test, mse_test, rmse_test, mape_test = evaluate_forecast(y_test_inverse, yhat_inverse)

res = pd.DataFrame([name, mae_test, mse_test, rmse_test, mape_test], index= ["Name", "mae_test", "mse_test", "rmse_test", "mape_test"]).T
df_results = df_results.append(res)

In [ ]:
df_results

In [ ]:
df_results.sort_values(by=['rmse_test'])